In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation, Dense, Flatten, BatchNormalization, Conv2D, MaxPool2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import categorical_crossentropy
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import confusion_matrix
import itertools
import os
import shutil
import random
import glob
import matplotlib.pyplot as plt
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [ ]:
os.chdir('data/cats-vs-dogs')
if os.path.isdir('train/dog') is False:
  os.makedirs('train/cat')
  os.makedirs('train/dog')
  os.makedirs('valid/cat')
  os.makedirs('valid/dog')
  os.makedirs('test/cat')
  os.makedirs('test/dog')

  for c in random.sample(glob.glob('train/cat.*'), 500):
    shutil.move(c, 'train/cat')
  for c in random.sample(glob.glob('train/dog*'), 500):
    shutil.move(c, 'train/dog')
  for c in random.sample(glob.glob('train/cat*'), 100):
    shutil.move(c, 'valid/cat')
  for c in random.sample(glob.glob('train/dog*'), 100):
    shutil.move(c, 'valid/dog')
  for c in random.sample(glob.glob('train/cat*'), 50):
    shutil.move(c, 'test/cat')
  for c in random.sample(glob.glob('train/dog*'), 50):
    shutil.move(c, 'test/dog')

os.chdir('../../')

In [ ]:
train_path = 'data/cats-vs-dogs/train'
valid_path = 'data/cats-vs-dogs/valid'
test_path  = 'data/cats-vs-dogs/test'

In [ ]:
train_batches = ImageDataGenerator(preprocessing_function=tf.keras.applications.vgg16.preprocess_input) \
  .flow_from_directory(directory=train_path, target_size=(244,244), classes=['cat', 'dog'], batch_size=10)
valid_batches = ImageDataGenerator(preprocessing_function=tf.keras.applications.vgg16.preprocess_input) \
  .flow_from_directory(directory=valid_path, target_size=(244,244), classes=['cat', 'dog'], batch_size=10)
test_batches = ImageDataGenerator(preprocessing_function=tf.keras.applications.vgg16.preprocess_input) \
  .flow_from_directory(directory=test_path, target_size=(244,244), classes=['cat', 'dog'], batch_size=10, shuffle=False)

In [ ]:
imgs, labels = next(train_batches)

In [ ]:
def plotImages(images_arr):
  fig, axes = plt.subplots(1,10, figsize=(20,20))
  axes = axes.flatten()
  for img, ax in zip(images_arr, axes):
    ax.imshow(img)
    ax.axis('off')
  plt.tight_layout()
  plt.show()

In [ ]:
plotImages(imgs)
print(labels)

In [ ]:
model = Sequential([
  Conv2D(filters=32, kernel_size=(3,3), activation='relu', padding='same', input_shape=(244,244,3)),
  MaxPool2D(pool_size=(2,2), strides=2),
  Conv2D(filters=64, kernel_size=(3,3), activation='relu', padding='same'),
  MaxPool2D(pool_size=(2,2), strides=2),
  Flatten(),
  Dense(units=2, activation='softmax'), # units relates to the number of classes used
])

In [ ]:
model.summary()

In [ ]:
model.compile(
  optimizer=Adam(learning_rate=0.0001),
  loss='categorical_crossentropy',
  metrics=['accuracy']
)

In [ ]:
model.fit(x=train_batches, validation_data=valid_batches, epochs=10, verbose=2)

In [ ]:
if os.path.isfile('models/cats_and_dogs.h5') is False:
  model.save('models/cats_and_dogs.h5')

# Prediction

In [ ]:
test_images, test_labels = next(test_batches)
plotImages(test_images)
print(test_labels)

In [ ]:
test_batches.classes

In [ ]:
predictions = model.predict(x=test_batches, verbose=0)

In [ ]:
np.round(predictions)

In [ ]:
cm = confusion_matrix(y_true=test_batches.classes, y_pred=np.argmax(predictions, axis=-1))

In [ ]:
def plot_confusion_matrix(cm, classes, normalize=False, title="Confusion matrix", cmap=plt.cm.Blues):
  plt.imshow(cm, interpolation='nearest', cmap=cmap)
  plt.title(title)
  plt.colorbar()
  tick_marks = np.arange(len(classes))
  plt.xticks(tick_marks, classes)
  plt.yticks(tick_marks, classes)

  if normalize:
    cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
    print("Noramlized confusion matrix")
  else:
    print("Confusion matrix, without normalization")
  
  print(cm)

  thresh = cm.max() / 2.
  for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
    plt.text(j, i, cm[i,j], horizontalalignment='center', color='white' if cm[j, i] > thresh else "black")

  plt.tight_layout()
  plt.ylabel("True label")
  plt.xlabel("Predicted label")


In [ ]:
cm_plot_labels = ['cat', 'dog']
plot_confusion_matrix(cm=cm, classes=cm_plot_labels)

# VGG16 Model transfer learning - fine-tuning

In [ ]:
vgg16_model = tf.keras.applications.vgg16.VGG16()

In [ ]:
vgg16_model.summary()

In [ ]:
model = Sequential()
for layer in vgg16_model.layers[:-1]:
  model.add(layer)

In [ ]:
model.summary()

In [ ]:
for layer in model.layers:
  layer.trainable = False

In [ ]:
model.add(Dense(units=2, activation='softmax'))

In [ ]:
model.summary()

In [ ]:
imgs, labels = next(train_batches)

In [ ]:
model.compile(
  optimizer=Adam(learning_rate=0.0001),
  loss='categorical_crossentropy',
  metrics=['accuracy']
)

In [ ]:
model.fit(x=train_batches, validation_data=valid_batches, epochs=5, verbose=2)

In [ ]:
test_imgs, test_labels = next(test_batches)
plotImages(test_images)
print(test_labels)

In [ ]:
predictions = model.predict(x=test_batches, verbose=0)

In [ ]:
np.round(predictions)

In [ ]:
cm = confusion_matrix(y_true=test_batches.classes, y_pred=np.argmax(predictions, axis=-1))

In [ ]:
cm_labels = ['cat', 'dog']
plot_confusion_matrix(cm=cm, classes=cm_labels)

# MobileNet transfer learning

In [ ]:
mobile = tf.keras.applications.mobilenet.MobileNet()